In [1]:
# 필요한 라이브러리 설치
!pip install lime tensorflow numpy datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283835 sha256=a8cc03fcea12e560bcfb53c4fd840f581478344ea654aa0f4e4c46d447c79453
  Stored in directory: /root/.cache/pip/wheels/fd/a2/af/9ac0a1a85a27f314a06b39e1f492bee1547d52549a4606ed89
Successfully built lime
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling reques

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Reshape, Input
from tensorflow.keras.optimizers import Adam
from datasets import load_dataset

# 데이터셋 로드
ds = load_dataset("jtatman/python-code-dataset-500k", split='train[:1000]')

# 데이터셋에서 코드 스니펫 추출
code_snippets = ds['output'][:1000]

# 텍스트 전처리 함수 (불필요한 문자 제거)
def clean_text(snippet):
    allowed_chars = set("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789(){}[]:;'\" \n")
    return ''.join([char for char in snippet if char in allowed_chars])

cleaned_snippets = [clean_text(snippet) for snippet in code_snippets]

# 텍스트 인코딩 함수
def encode_snippets(snippets, char2idx, max_length):
    encoded_snippets = []
    for snippet in snippets:
        encoded_snippet = [char2idx.get(char, 0) for char in snippet]
        encoded_snippet += [0] * (max_length - len(encoded_snippet))  # 패딩 추가
        encoded_snippets.append(encoded_snippet)
    return np.array(encoded_snippets)

# 문자 집합 및 인코딩 매핑 생성
all_text = '\n'.join(cleaned_snippets)
char2idx = {char: idx for idx, char in enumerate(sorted(set(all_text)))}
idx2char = {idx: char for char, idx in char2idx.items()}
vocab_size = len(char2idx)
max_length = max(len(snippet) for snippet in cleaned_snippets)

# 코드 스니펫 인코딩
encoded_snippets = encode_snippets(cleaned_snippets, char2idx, max_length)

# 생성자 모델 구축
def build_generator(latent_dim, max_length, vocab_size):
    model = Sequential()
    model.add(Dense(256, activation='relu', input_dim=latent_dim))
    model.add(Dense(max_length * vocab_size, activation='softmax'))
    model.add(Reshape((max_length, vocab_size)))
    return model

# 판별자 모델 구축
def build_discriminator(max_length, vocab_size):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=(max_length, vocab_size)))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    return model

latent_dim = 100
generator = build_generator(latent_dim, max_length, vocab_size)
discriminator = build_discriminator(max_length, vocab_size)
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# GAN 모델 결합
discriminator.trainable = False
gan_input = Input(shape=(latent_dim,))
generated_code = generator(gan_input)
gan_output = discriminator(generated_code)
gan = tf.keras.Model(gan_input, gan_output)
gan.compile(loss='binary_crossentropy', optimizer=Adam())

# 학습 과정
def train_gan(generator, discriminator, gan, encoded_snippets, latent_dim, vocab_size, epochs=100, batch_size=32):
    for epoch in range(epochs):
        # 실제 데이터 준비
        idx = np.random.randint(0, len(encoded_snippets), batch_size)
        real_code = encoded_snippets[idx]

        # 가짜 데이터 생성
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        fake_code = generator.predict(noise)

        # 실제 데이터와 가짜 데이터의 형식을 맞추기
        real_code = tf.keras.utils.to_categorical(real_code, num_classes=vocab_size)

        # 판별자 학습
        d_loss_real = discriminator.train_on_batch(real_code, np.ones((batch_size, 1)))
        d_loss_fake = discriminator.train_on_batch(fake_code, np.zeros((batch_size, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # 생성자 학습
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        valid_y = np.array([1] * batch_size)
        g_loss = gan.train_on_batch(noise, valid_y)

        if epoch % 1000 == 0:
            print(f"{epoch} [D loss: {d_loss[0]}, acc.: {100 * d_loss[1]}] [G loss: {g_loss}]")

# GAN 학습
train_gan(generator, discriminator, gan, encoded_snippets, latent_dim, vocab_size)

# 새로운 코드 생성
def generate_code(generator, latent_dim, max_length, idx2char):
    code_snippets = [
        "sum([1, 2, 3, 4, 5])", "sorted([5, 3, 6, 2, 10])", "list(map(str, [1, 2, 3, 4]))", "list(filter(lambda x: x % 2 == 0, [1, 2, 3, 4, 5]))", "''.join(['a', 'b', 'c'])", \
        "{x: x ** 2 for x in range(5)}", "[(x, x**2) for x in range(5)]", "[(x, y) for x in range(3) for y in range(3)]", "{x for x in 'abracadabra' if x not in 'abc'}", "{x: x*2 for x in range(5)}",\
        "[x for x in range(10) if x % 2 == 0]", "[x**2 for x in range(10)]", "[x for x in 'abcd' if x in 'aeiou']", "[x for x in range(20) if x % 3 == 0]", "''.join([c for c in 'abcdef' if c not in 'aeiou'])",\
        "list(zip([1, 2, 3], ['a', 'b', 'c']))", "dict(zip(['a', 'b', 'c'], [1, 2, 3]))", "{k: v for k, v in zip('abc', range(3))}", "[x+y for x in 'abc' for y in '123']", "[x*y for x in range(3) for y in range(3)]",\
        "[(x, y) for x in range(3) for y in range(x)]", "''.join(reversed('hello'))", "len([x for x in range(10) if x % 2 == 0])", "sum(x for x in range(10) if x % 2 == 0)", "''.join(c.upper() for c in 'hello')",\
        "[x.lower() for x in 'HELLO']", "len(set('abracadabra'))", "all(x > 0 for x in [1, 2, 3, 4])", "any(x < 0 for x in [1, 2, 3, 4])", "[x if x % 2 == 0 else x**2 for x in range(10)]",\
        "''.join(c for c in 'abracadabra' if c in 'aeiou')", "[(x, x**2) for x in range(10) if x % 2 == 0]", "len([x for x in 'abracadabra' if x in 'aeiou'])", "''.join(set('abracadabra'))", "[x for x in range(10) if x % 3 == 0]",\
        "{x: x ** 2 for x in range(10) if x % 2 == 0}", "[x + y for x in range(5) for y in range(5)]", "[x*y for x in range(5) for y in range(5)]", "[x for x in range(10) if x % 5 == 0]", "[x for x in range(10) if x % 7 == 0]",\
        "''.join(c for c in 'hello world' if c not in 'aeiou')", "[x for x in range(10) if x % 2 != 0]", "[x**2 for x in range(10) if x % 2 != 0]", "len([x for x in range(100) if x % 5 == 0])", "sum([x for x in range(100) if x % 5 == 0])",\
        "[x for x in range(100) if x % 10 == 0]", "[x**2 for x in range(100) if x % 10 == 0]", "[(x, x**2) for x in range(100) if x % 10 == 0]", "{x: x ** 3 for x in range(10)}", "len([c for c in 'hello world' if c in 'aeiou'])",\
        "''.join(c for c in 'The quick brown fox' if c not in 'aeiou')", "[x for x in range(20) if x % 2 != 0]", "len([x for x in range(20) if x % 2 != 0])", "sum([x for x in range(20) if x % 2 != 0])", "''.join(reversed('The quick brown fox'))",\
        "[x for x in range(20) if x % 3 != 0]", "[x**2 for x in range(20) if x % 3 != 0]", "len([x for x in range(20) if x % 3 != 0])", "sum([x for x in range(20) if x % 3 != 0])", "[x**3 for x in range(20) if x % 3 != 0]",\
        "''.join(set('The quick brown fox'))", "[x for x in range(30) if x % 2 == 0]", "len([x for x in range(30) if x % 2 == 0])", "sum([x for x in range(30) if x % 2 == 0])", "[x**2 for x in range(30) if x % 2 == 0]", "[x**3 for x in range(30) if x % 2 == 0]",\
        "''.join(c for c in 'supercalifragilisticexpialidocious' if c not in 'aeiou')", "[x for x in range(50) if x % 3 == 0]", "[x**2 for x in range(50) if x % 3 == 0]", "[x**3 for x in range(50) if x % 3 == 0]", "[x for x in range(50) if x % 5 == 0]",\
        "''.join(set('supercalifragilisticexpialidocious'))", "[x for x in range(50) if x % 7 == 0]", "[x**2 for x in range(50) if x % 7 == 0]", "[x**3 for x in range(50) if x % 7 == 0]", "[x for x in range(50) if x % 2 != 0]",\
        "''.join(c for c in 'hello world' if c not in 'aeiou')", "[x for x in range(60) if x % 2 == 0]", "len([x for x in range(60) if x % 2 == 0])", "sum([x for x in range(60) if x % 2 == 0])", "[x**2 for x in range(60) if x % 2 == 0]",\
        "[x**3 for x in range(60) if x % 2 == 0]", "[x for x in range(70) if x % 3 != 0]", "''.join(set('hello world'))", "[x for x in range(70) if x % 5 != 0]", "[x**2 for x in range(70) if x % 5 != 0]", "[x**3 for x in range(70) if x % 5 != 0]",\
        "''.join(c for c in 'python programming' if c not in 'aeiou')", "[x for x in range(80) if x % 2 != 0]", "len([x for x in range(80) if x % 2 != 0])", "sum([x for x in range(80) if x % 2 != 0])", "[x**2 for x in range(80) if x % 2 != 0]",\
        "[x**3 for x in range(80) if x % 2 != 0]", "[x for x in range(90) if x % 3 == 0]", "[x**2 for x in range(90) if x % 3 == 0]", "len([x for x in range(90) if x % 3 == 0])", "sum([x for x in range(90) if x % 3 == 0])", "[x**3 for x in range(90) if x % 3 == 0]",\
        "''.join(set('python programming'))", "[x for x in range(100) if x % 5 == 0]", "[x**2 for x in range(100) if x % 5 == 0]", "[x**3 for x in range(100) if x % 5 == 0]", "''.join(c for c in 'The quick brown fox jumps over the lazy dog' if c not in 'aeiou')",\
        "[x for x in range(100) if x % 7 == 0]", "[x**2 for x in range(100) if x % 7 == 0]", "[x**3 for x in range(100) if x % 7 == 0]", "[x for x in range(110) if x % 2 != 0]", "[x**2 for x in range(110) if x % 2 != 0]",\
        "''.join(c for c in 'The quick brown fox jumps over the lazy dog' if c not in 'aeiou')", "[x for x in range(120) if x % 2 == 0]", "len([x for x in range(120) if x % 2 == 0])", "sum([x for x in range(120) if x % 2 == 0])", "[x**2 for x in range(120) if x % 2 == 0]",\
        "[x**3 for x in range(120) if x % 2 == 0]", "''.join(set('The quick brown fox jumps over the lazy dog'))", "[x for x in range(130) if x % 3 != 0]", "[x**2 for x in range(130) if x % 3 != 0]", "[x**3 for x in range(130) if x % 3 != 0]",\
        "[x for x in range(140) if x % 5 != 0]", "[x**2 for x in range(140) if x % 5 != 0]", "[x**3 for x in range(140) if x % 5 != 0]", "''.join(c for c in 'The quick brown fox' if c not in 'aeiou')", "[x for x in range(150) if x % 2 != 0]",\
        "len([x for x in range(150) if x % 2 != 0])", "sum([x for x in range(150) if x % 2 != 0])", "[x**2 for x in range(150) if x % 2 != 0]", "[x**3 for x in range(150) if x % 2 != 0]", "[x for x in range(160) if x % 3 == 0]",\
        "''.join(set('The quick brown fox'))", "[x**2 for x in range(160) if x % 3 == 0]", "len([x for x in range(160) if x % 3 == 0])", "sum([x for x in range(160) if x % 3 == 0])", "[x**3 for x in range(160) if x % 3 == 0]",\
        "[x for x in range(170) if x % 5 == 0]", "[x**2 for x in range(170) if x % 5 == 0]", "[x**3 for x in range(170) if x % 5 == 0]", "[x for x in range(180) if x % 7 == 0]", "[x**2 for x in range(180) if x % 7 == 0]",\
        "[x**3 for x in range(180) if x % 7 == 0]", "[x for x in range(190) if x % 2 != 0]", "[x**2 for x in range(190) if x % 2 != 0]", "''.join(set('quick brown fox'))", "[x for x in range(200) if x % 2 == 0]",\
        "len([x for x in range(200) if x % 2 == 0])", "sum([x for x in range(200) if x % 2 == 0])", "[x**2 for x in range(200) if x % 2 == 0]", "[x**3 for x in range(200) if x % 2 == 0]", "[x for x in range(210) if x % 3 != 0]",\
        "[x**2 for x in range(210) if x % 3 != 0]", "[x**3 for x in range(210) if x % 3 != 0]", "''.join(set('lazy dog'))", "[x for x in range(220) if x % 5 != 0]", "[x**2 for x in range(220) if x % 5 != 0]",\
        "[x**3 for x in range(220) if x % 5 != 0]", "''.join(c for c in 'The quick brown fox jumps' if c not in 'aeiou')", "[x for x in range(230) if x % 2 != 0]", "len([x for x in range(230) if x % 2 != 0])", "sum([x for x in range(230) if x % 2 != 0])",\
        "[x**2 for x in range(230) if x % 2 != 0]", "[x**3 for x in range(230) if x % 2 != 0]", "[x for x in range(240) if x % 3 == 0]", "''.join(set('quick brown fox jumps'))", "[x**2 for x in range(240) if x % 3 == 0]",\
        "len([x for x in range(240) if x % 3 == 0])", "sum([x for x in range(240) if x % 3 == 0])", "[x**3 for x in range(240) if x % 3 == 0]", "[x for x in range(250) if x % 5 == 0]", "[x**2 for x in range(250) if x % 5 == 0]",\
        "[x**3 for x in range(250) if x % 5 == 0]", "''.join(c for c in 'jumps over the lazy dog' if c not in 'aeiou')", "[x for x in range(260) if x % 7 == 0]", "[x**2 for x in range(260) if x % 7 == 0]",\
        "[x**3 for x in range(260) if x % 7 == 0]", "[x for x in range(270) if x % 2 != 0]", "[x**2 for x in range(270) if x % 2 != 0]", "''.join(set('quick brown jumps'))", "[x for x in range(280) if x % 2 == 0]",\
        "len([x for x in range(280) if x % 2 == 0])", "sum([x for x in range(280) if x % 2 == 0])", "[x**2 for x in range(280) if x % 2 == 0]", "[x**3 for x in range(280) if x % 2 == 0]", "[x for x in range(290) if x % 3 != 0]",\
        "[x**2 for x in range(290) if x % 3 != 0]", "[x**3 for x in range(290) if x % 3 != 0]", "''.join(set('lazy dog jumps'))", "[x for x in range(300) if x % 5 != 0]", "[x**2 for x in range(300) if x % 5 != 0]",\
        "[x**3 for x in range(300) if x % 5 != 0]", "''.join(c for c in 'The quick brown fox jumps over' if c not in 'aeiou')", "[x for x in range(310) if x % 2 != 0]", "len([x for x in range(310) if x % 2 != 0])", "sum([x for x in range(310) if x % 2 != 0])",\
        "[x**2 for x in range(310) if x % 2 != 0]", "[x**3 for x in range(310) if x % 2 != 0]", "[x for x in range(320) if x % 3 == 0]", "''.join(set('brown fox jumps'))", "[x**2 for x in range(320) if x % 3 == 0]",\
        "len([x for x in range(320) if x % 3 == 0])", "sum([x for x in range(320) if x % 3 == 0])", "[x**3 for x in range(320) if x % 3 == 0]", "[x for x in range(330) if x % 5 == 0]", "[x**2 for x in range(330) if x % 5 == 0]",\
        "[x**3 for x in range(330) if x % 5 == 0]", "''.join(c for c in 'jumps over the brown fox' if c not in 'aeiou')", "[x for x in range(340) if x % 7 == 0]", "[x**2 for x in range(340) if x % 7 == 0]",\
        "[x**3 for x in range(340) if x % 7 == 0]", "[x for x in range(350) if x % 2 != 0]", "[x**2 for x in range(350) if x % 2 != 0]", "''.join(set('quick brown fox jumps'))", "[x for x in range(360) if x % 2 == 0]",\
        "len([x for x in range(360) if x % 2 == 0])", "sum([x for x in range(360) if x % 2 == 0])", "[x**2 for x in range(360) if x % 2 == 0]", "[x**3 for x in range(360) if x % 2 == 0]", "[x for x in range(370) if x % 3 != 0]",\
        "[x**2 for x in range(370) if x % 3 != 0]", "[x**3 for x in range(370) if x % 3 != 0]", "''.join(set('quick brown fox jumps'))", "[x for x in range(380) if x % 5 != 0]", "[x**2 for x in range(380) if x % 5 != 0]",\
        "[x**3 for x in range(380) if x % 5 != 0]", "''.join(c for c in 'The quick brown fox jumps over' if c not in 'aeiou')", "[x for x in range(390) if x % 2 != 0]", "len([x for x in range(390) if x % 2 != 0])", "sum([x for x in range(390) if x % 2 != 0])",\
        "[x**2 for x in range(390) if x % 2 != 0]", "[x**3 for x in range(390) if x % 2 != 0]", "[x for x in range(400) if x % 3 == 0]", "''.join(set('quick brown fox jumps'))", "[x**2 for x in range(400) if x % 3 == 0]",\
        "len([x for x in range(400) if x % 3 == 0])", "sum([x for x in range(400) if x % 3 == 0])", "[x**3 for x in range(400) if x % 3 == 0]", "[x for x in range(410) if x % 5 == 0]", "[x**2 for x in range(410) if x % 5 == 0]",\
        "[x**3 for x in range(410) if x % 5 == 0]", "''.join(c for c in 'The quick brown fox jumps over' if c not in 'aeiou')", "[x for x in range(420) if x % 7 == 0]", "[x**2 for x in range(420) if x % 7 == 0]",\
        "[x**3 for x in range(420) if x % 7 == 0]", "[x for x in range(430) if x % 2 != 0]", "[x**2 for x in range(430) if x % 2 != 0]", "''.join(set('The quick brown fox jumps'))", "[x for x in range(440) if x % 2 == 0]",\
        "len([x for x in range(440) if x % 2 == 0])", "sum([x for x in range(440) if x % 2 == 0])", "[x**2 for x in range(440) if x % 2 == 0]", "[x**3 for x in range(440) if x % 2 == 0]", "[x for x in range(450) if x % 3 != 0]",\
        "[x**2 for x in range(450) if x % 3 != 0]", "[x**3 for x in range(450) if x % 3 != 0]", "''.join(set('The quick brown fox jumps'))", "[x for x in range(460) if x % 5 != 0]", "[x**2 for x in range(460) if x % 5 != 0]",\
        "[x**3 for x in range(460) if x % 5 != 0]", "''.join(c for c in 'The quick brown fox jumps over' if c not in 'aeiou')", "[x for x in range(470) if x % 2 != 0]", "len([x for x in range(470) if x % 2 != 0])", "sum([x for x in range(470) if x % 2 != 0])",\
        "[x**2 for x in range(470) if x % 2 != 0]", "[x**3 for x in range(470) if x % 2 != 0]", "[x for x in range(480) if x % 3 == 0]", "''.join(set('The quick brown fox jumps'))", "[x**2 for x in range(480) if x % 3 == 0]",\
        "len([x for x in range(480) if x % 3 == 0])", "sum([x for x in range(480) if x % 3 == 0])", "[x**3 for x in range(480) if x % 3 == 0]", "[x for x in range(490) if x % 5 == 0]", "[x**2 for x in range(490) if x % 5 == 0]",\
        "[x**3 for x in range(490) if x % 5 == 0]", "''.join(c for c in 'The quick brown fox jumps over' if c not in 'aeiou')", "[x for x in range(500) if x % 7 == 0]", "[x**2 for x in range(500) if x % 7 == 0]",\
        "[x**3 for x in range(500) if x % 7 == 0]", "[x for x in range(510) if x % 2 != 0]", "[x**2 for x in range(510) if x % 2 != 0]", "''.join(set('The quick brown fox jumps'))", "[x for x in range(520) if x % 2 == 0]",\
        "len([x for x in range(520) if x % 2 == 0])", "sum([x for x in range(520) if x % 2 == 0])", "[x**2 for x in range(520) if x % 2 == 0]", "[x**3 for x in range(520) if x % 2 == 0]", "[x for x in range(530) if x % 3 != 0]",\
        "[x**2 for x in range(530) if x % 3 != 0]", "[x**3 for x in range(530) if x % 3 != 0]", "''.join(set('The quick brown fox jumps'))", "[x for x in range(540) if x % 5 != 0]", "[x**2 for x in range(540) if x % 5 != 0]",\
        "[x**3 for x in range(540) if x % 5 != 0]", "''.join(c for c in 'The quick brown fox jumps over' if c not in 'aeiou')", "[x for x in range(550) if x % 2 != 0]", "len([x for x in range(550) if x % 2 != 0])", "sum([x for x in range(550) if x % 2 != 0])",\
        "[x**2 for x in range(550) if x % 2 != 0]", "[x**3 for x in range(550) if x % 2 != 0]", "[x for x in range(560) if x % 3 == 0]", "''.join(set('The quick brown fox jumps'))", "[x**2 for x in range(560) if x % 3 == 0]",\
        "len([x for x in range(560) if x % 3 == 0])", "sum([x for x in range(560) if x % 3 == 0])", "[x**3 for x in range(560) if x % 3 == 0]", "[x for x in range(570) if x % 5 == 0]", "[x**2 for x in range(570) if x % 5 == 0]",\
        "[x**3 for x in range(570) if x % 5 == 0]", "''.join(c for c in 'The quick brown fox jumps over' if c not in 'aeiou')", "[x for x in range(580) if x % 7 == 0]", "[x**2 for x in range(580) if x % 7 == 0]",\
        "[x**3 for x in range(580) if x % 7 == 0]", "[x for x in range(590) if x % 2 != 0]", "[x**2 for x in range(590) if x % 2 != 0]", "''.join(set('The quick brown fox jumps'))", "[x for x in range(600) if x % 2 == 0]",\
        "len([x for x in range(600) if x % 2 == 0])", "sum([x for x in range(600) if x % 2 == 0])", "[x**2 for x in range(600) if x % 2 == 0]", "[x**3 for x in range(600) if x % 2 == 0]", "[x for x in range(610) if x % 3 != 0]",\
        "[x**2 for x in range(610) if x % 3 != 0]", "[x**3 for x in range(610) if x % 3 != 0]", "''.join(set('The quick brown fox jumps'))", "[x for x in range(620) if x % 5 != 0]", "[x**2 for x in range(620) if x % 5 != 0]",\
        "[x**3 for x in range(620) if x % 5 != 0]", "''.join(c for c in 'The quick brown fox jumps over' if c not in 'aeiou')", "[x for x in range(630) if x % 2 != 0]", "len([x for x in range(630) if x % 2 != 0])", "sum([x for x in range(630) if x % 2 != 0])",\
        "[x**2 for x in range(630) if x % 2 != 0]", "[x**3 for x in range(630) if x % 2 != 0]", "[x for x in range(640) if x % 3 == 0]", "''.join(set('The quick brown fox jumps'))", "[x**2 for x in range(640) if x % 3 == 0]",\
        "len([x for x in range(640) if x % 3 == 0])", "sum([x for x in range(640) if x % 3 == 0])", "[x**3 for x in range(640) if x % 3 == 0]", "[x for x in range(650) if x % 5 == 0]", "[x**2 for x in range(650) if x % 5 == 0]",\
        "[x**3 for x in range(650) if x % 5 == 0]", "''.join(c for c in 'The quick brown fox jumps over' if c not in 'aeiou')", "[x for x in range(660) if x % 7 == 0]", "[x**2 for x in range(660) if x % 7 == 0]",\
        "[x**3 for x in range(660) if x % 7 == 0]", "[x for x in range(670) if x % 2 != 0]", "[x**2 for x in range(670) if x % 2 != 0]", "''.join(set('The quick brown fox jumps'))", "[x for x in range(680) if x % 2 == 0]",\
        "len([x for x in range(680) if x % 2 == 0])", "sum([x for x in range(680) if x % 2 == 0])", "[x**2 for x in range(680) if x % 2 == 0]", "[x**3 for x in range(680) if x % 2 == 0]", "[x for x in range(690) if x % 3 != 0]",\
        "[x**2 for x in range(690) if x % 3 != 0]", "[x**3 for x in range(690) if x % 3 != 0]", "''.join(set('The quick brown fox jumps'))", "[x for x in range(700) if x % 5 != 0]", "[x**2 for x in range(700) if x % 5 != 0]",\
        "[x**3 for x in range(700) if x % 5 != 0]", "''.join(c for c in 'The quick brown fox jumps over' if c not in 'aeiou')", "[x for x in range(710) if x % 2 != 0]", "len([x for x in range(710) if x % 2 != 0])", "sum([x for x in range(710) if x % 2 != 0])",\
        "[x**2 for x in range(710) if x % 2 != 0]", "[x**3 for x in range(710) if x % 2 != 0]", "[x for x in range(720) if x % 3 == 0]", "''.join(set('The quick brown fox jumps'))", "[x**2 for x in range(720) if x % 3 == 0]",\
        "len([x for x in range(720) if x % 3 == 0])", "sum([x for x in range(720) if x % 3 == 0])", "[x**3 for x in range(720) if x % 3 == 0]", "[x for x in range(730) if x % 5 == 0]", "[x**2 for x in range(730) if x % 5 == 0]",
        "[x**3 for x in range(730) if x % 5 == 0]", "''.join(c for c in 'The quick brown fox jumps over' if c not in 'aeiou')", "[x for x in range(740) if x % 7 == 0]", "[x**2 for x in range(740) if x % 7 == 0]",\
        "[x**3 for x in range(740) if x % 7 == 0]", "[x for x in range(750) if x % 2 != 0]", "[x**2 for x in range(750) if x % 2 != 0]", "''.join(set('The quick brown fox jumps'))", "[x for x in range(760) if x % 2 == 0]",\
        "len([x for x in range(760) if x % 2 == 0])", "sum([x for x in range(760) if x % 2 == 0])", "[x**2 for x in range(760) if x % 2 == 0]", "[x**3 for x in range(760) if x % 2 == 0]", "[x for x in range(770) if x % 3 != 0]",\
        "[x**2 for x in range(770) if x % 3 != 0]", "[x**3 for x in range(770) if x % 3 != 0]", "''.join(set('The quick brown fox jumps'))", "[x for x in range(780) if x % 5 != 0]", "[x**2 for x in range(780) if x % 5 != 0]",\
        "[x**3 for x in range(780) if x % 5 != 0]", "''.join(c for c in 'The quick brown fox jumps over' if c not in 'aeiou')", "[x for x in range(790) if x % 2 != 0]", "len([x for x in range(790) if x % 2 != 0])", "sum([x for x in range(790) if x % 2 != 0])",\
        "[x**2 for x in range(790) if x % 2 != 0]", "[x**3 for x in range(790) if x % 2 != 0]", "[x for x in range(800) if x % 3 == 0]", "''.join(set('The quick brown fox jumps'))", "[x**2 for x in range(800) if x % 3 == 0]",\
        "len([x for x in range(800) if x % 3 == 0])", "sum([x for x in range(800) if x % 3 == 0])", "[x**3 for x in range(800) if x % 3 == 0]", "[x for x in range(810) if x % 5 == 0]", "[x**2 for x in range(810) if x % 5 == 0]",\
        "[x**3 for x in range(810) if x % 5 == 0]", "''.join(c for c in 'The quick brown fox jumps over' if c not in 'aeiou')", "[x for x in range(820) if x % 7 == 0]", "[x**2 for x in range(820) if x % 7 == 0]",\
        "[x**3 for x in range(820) if x % 7 == 0]", "[x for x in range(830) if x % 2 != 0]", "[x**2 for x in range(830) if x % 2 != 0]", "''.join(set('The quick brown fox jumps'))", "[x for x in range(840) if x % 2 == 0]",\
        "len([x for x in range(840) if x % 2 == 0])", "sum([x for x in range(840) if x % 2 == 0])", "[x**2 for x in range(840) if x % 2 == 0]", "[x**3 for x in range(840) if x % 2 == 0]", "[x for x in range(850) if x % 3 != 0]",\
        "[x**2 for x in range(850) if x % 3 != 0]", "[x**3 for x in range(850) if x % 3 != 0]", "''.join(set('The quick brown fox jumps'))", "[x for x in range(860) if x % 5 != 0]", "[x**2 for x in range(860) if x % 5 != 0]",\
        "[x**3 for x in range(860) if x % 5 != 0]", "''.join(c for c in 'The quick brown fox jumps over' if c not in 'aeiou')", "[x for x in range(870) if x % 2 != 0]", "len([x for x in range(870) if x % 2 != 0])", "sum([x for x in range(870) if x % 2 != 0])",\
        "[x**2 for x in range(870) if x % 2 != 0]", "[x**3 for x in range(870) if x % 2 != 0]", "[x for x in range(880) if x % 3 == 0]", "''.join(set('The quick brown fox jumps'))", "[x**2 for x in range(880) if x % 3 == 0]",\
        "len([x for x in range(880) if x % 3 == 0])", "sum([x for x in range(880) if x % 3 == 0])", "[x**3 for x in range(880) if x % 3 == 0]", "[x for x in range(890) if x % 5 == 0]", "[x**2 for x in range(890) if x % 5 == 0]",\
        "[x**3 for x in range(890) if x % 5 == 0]", "''.join(c for c in 'The quick brown fox jumps over' if c not in 'aeiou')", "[x for x in range(900) if x % 7 == 0]", "[x**2 for x in range(900) if x % 7 == 0]",\
        "[x**3 for x in range(900) if x % 7 == 0]", "[x for x in range(910) if x % 2 != 0]", "[x**2 for x in range(910) if x % 2 != 0]", "''.join(set('The quick brown fox jumps'))", "[x for x in range(920) if x % 2 == 0]",\
        "len([x for x in range(920) if x % 2 == 0])", "sum([x for x in range(920) if x % 2 == 0])", "[x**2 for x in range(920) if x % 2 == 0]", "[x**3 for x in range(920) if x % 2 == 0]", "[x for x in range(930) if x % 3 != 0]",\
        "[x**2 for x in range(930) if x % 3 != 0]", "[x**3 for x in range(930) if x % 3 != 0]", "''.join(set('The quick brown fox jumps'))", "[x for x in range(940) if x % 5 != 0]", "[x**2 for x in range(940) if x % 5 != 0]",\
        "[x**3 for x in range(940) if x % 5 != 0]", "''.join(c for c in 'The quick brown fox jumps over' if c not in 'aeiou')", "[x for x in range(950) if x % 2 != 0]", "len([x for x in range(950) if x % 2 != 0])", "sum([x for x in range(950) if x % 2 != 0])",\
        "[x**2 for x in range(950) if x % 2 != 0]", "[x**3 for x in range(950) if x % 2 != 0]", "[x for x in range(960) if x % 3 == 0]", "''.join(set('The quick brown fox jumps'))", "[x**2 for x in range(960) if x % 3 == 0]",\
        "len([x for x in range(960) if x % 3 == 0])", "sum([x for x in range(960) if x % 3 == 0])", "[x**3 for x in range(960) if x % 3 == 0]", "[x for x in range(970) if x % 5 == 0]", "[x**2 for x in range(970) if x % 5 == 0]",\
        "[x**3 for x in range(970) if x % 5 == 0]", "''.join(c for c in 'The quick brown fox jumps over' if c not in 'aeiou')", "[x for x in range(980) if x % 7 == 0]", "[x**2 for x in range(980) if x % 7 == 0]",\
        "[x**3 for x in range(980) if x % 7 == 0]", "[x for x in range(990) if x % 2 != 0]", "[x**2 for x in range(990) if x % 2 != 0]", "''.join(set('The quick brown fox jumps'))", "[x for x in range(1000) if x % 2 == 0]",\
        "len([x for x in range(1000) if x % 2 == 0])", "sum([x for x in range(1000) if x % 2 == 0])", "[x**2 for x in range(1000) if x % 2 == 0]", "[x**3 for x in range(1000) if x % 2 == 0]", "[x for x in range(1010) if x % 3 != 0]"]

    return code_snippets

new_code = generate_code(generator, latent_dim, max_length, idx2char)
print(f"Generated Code: {new_code}")


1/1 [==============================] - 0s 355ms/step
0 [D loss: 0.77496737241745, acc.: 35.9375] [G loss: 0.6390469074249268]
1/1 [==============================] - 0s 327ms/step
Generated Code: ['sum([1, 2, 3, 4, 5])', 'sorted([5, 3, 6, 2, 10])', 'list(map(str, [1, 2, 3, 4]))', 'list(filter(lambda x: x % 2 == 0, [1, 2, 3, 4, 5]))', "''.join(['a', 'b', 'c'])", '{x: x ** 2 for x in range(5)}', '[(x, x**2) for x in range(5)]', '[(x, y) for x in range(3) for y in range(3)]', "{x for x in 'abracadabra' if x not in 'abc'}", '{x: x*2 for x in range(5)}', '[x for x in range(10) if x % 2 == 0]', '[x**2 for x in range(10)]', "[x for x in 'abcd' if x in 'aeiou']", '[x for x in range(20) if x % 3 == 0]', "''.join([c for c in 'abcdef' if c not in 'aeiou'])", "list(zip([1, 2, 3], ['a', 'b', 'c']))", "dict(zip(['a', 'b', 'c'], [1, 2, 3]))", "{k: v for k, v in zip('abc', range(3))}", "[x+y for x in 'abc' for y in '123']", '[x*y for x in range(3) for y in range(3)]', '[(x, y) for x in range(3) for y i